# Model Inference on some CASME2 Clips 

In [8]:
import os

In [10]:
os.chdir('c:\\Users\\kings\\OneDrive - MMU\\MSC DATA SCIENCE\\MSC Project\\Msc-Project')

In [14]:
import numpy  as np
import pandas as pd
import csv


# visualization libraries
import matplotlib.pyplot as plt
import plotly.express as px

import cv2
import torch
import torch.nn.functional as F

# Live WebCam Demo Inception-V3

In [15]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 3)
state_dict = torch.load(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\models\Inception3_Adamproposed_augmediapipe.pt")

model.load_state_dict(state_dict)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

Using cache found in C:\Users\kings/.cache\torch\hub\pytorch_vision_v0.10.0


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [16]:
font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\datasets\CASME2\CASME2\CASME2-RAW\CASME2-RAW\sub01\EP19_06f.avi")
if not cap.isOpened():
    raise IOError("Cannot open webcam")

In [21]:
def predict_emotion(output):
    while True:
        # Capture a frame
        ret, frame = cap.read()
        facedet = cv2.CascadeClassifier(r'C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\Msc-Project\haarcascades\haarcascade_frontalface_default.xml')
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = facedet.detectMultiScale(gray_frame, 1.1, 4)

        if len(faces) == 0:
            print("No faces detected in the frame")
            continue  # Skip processing this frame if no faces are detected

        for x, y, w, h in faces:
            roi_gray = gray_frame[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            facess = facedet.detectMultiScale(roi_gray)
            if len(facess) == 0:
                print("Face not detected")
            else:
                for (ex, ey, ew, eh) in facess:
                    face_roi = roi_color[ey:ey+eh, ex:ex+ew]  # cropping the face

        rgb_image = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)
        image = cv2.resize(rgb_image, (299, 299))
        image = image / 255.0  # normalization
        data = torch.from_numpy(image)
        data = data.type(torch.FloatTensor)
        data = data.to(device)
        reshaped_data = data.permute(2, 0, 1)  # Reshape the tensor to have 3 as the first dimension
        reshaped_data = reshaped_data.unsqueeze(0)  # add a fourth dimension
        outputs = model(reshaped_data)
        pred = F.softmax(outputs[0], dim=-1)
        final_pred = torch.argmax(pred, 0)

        if final_pred == 0:
            emotion = "Negative"
        elif final_pred == 1:
            emotion = "Neutral"
        else:
            emotion = "Positive"

        text_x, text_y = x, y - 10  # Adjust the position above the bounding box
        cv2.putText(frame, emotion, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

        conf, classes = torch.max(pred, -1)
        emotion_id = [0, 1, 2]
        class_dict = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
        class_names = [class_dict[label] for label in emotion_id]
        result_data = []
        result_data.append([conf.item(), pred.tolist(), class_names[classes.item()]])

        # Write the result to a CSV file
        with open(output+'.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            if file.tell() == 0:
                writer.writerow(['Confidence', 'max_confidence', 'Emotion'])  # Write header only if the file is empty
            writer.writerows(result_data)

        cv2.imshow("Facial Expression Recognition", frame)

        if cv2.waitKey(2) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [20]:
predict_emotion('test')

Face not detected
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
Face not detected
Face not detected
Face not detected
Face not detected
Face not detected
Face not detected
Face not detected
No faces detected in the frame
Face not detected
Face not detected
Face not detected
No faces detected in the frame
No faces detected in the frame
Face not detected
Face not detected
Face not detected
Face not detected
Face not detected
No faces detected in the frame

In [7]:
df = pd.read_csv('sub01_EP19_06finception_mp.csv')

In [11]:
def plot_predictions(data, clip):
    df = data
    # Filter the data by emotions
    negative_data = df[df['Emotion'] == 'Negative']
    neutral_data = df[df['Emotion'] == 'Neutral']
    positive_data = df[df['Emotion'] == 'Positive']

    # Create a Plotly figure
    fig = px.line(df, x=df.index, y='Confidence', color='Emotion',
                labels={'index': 'Frame', 'Confidence': 'Confidence'},
                title=f'Confidence Scores Over Time for Different Emotions on {clip} clip')

    # Customize the figure appearance (optional)
    fig.update_traces(mode='lines+markers')
    fig.update_layout(legend_title='Emotion')
    fig.update_xaxes(title_text='Frame')
    fig.update_yaxes(title_text='Confidence')

    # # Show the interactive plot
    fig.show()


In [9]:
plot_predictions(df, 'sub01_EP19_06finception_facial_lanmarks')

In [22]:
cap = cv2.VideoCapture(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\datasets\CASME2\CASME2\CASME2-RAW\CASME2-RAW\sub26\EP07_37.avi")
if not cap.isOpened():
    raise IOError("Cannot open webcam")

In [23]:
predict_emotion('sub26_EP07_37')

In [33]:
df = pd.read_csv('sub26_EP07_37inception_mp.csv')

In [34]:
df['Confidence'].max()

0.9162644147872924

In [32]:
plot_predictions(df, 'sub26_EP07_37incpetion_fl')

In [6]:
cap = cv2.VideoCapture(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\datasets\CASME2\CASME2\CASME2-RAW\CASME2-RAW\sub14\EP09_06.avi")
if not cap.isOpened():
    raise IOError("Cannot open webcam")

In [7]:
predict_emotion('sub14_EP09_06')

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [16]:
df = pd.read_csv('sub14_EP09_06inception_mp.csv')

In [17]:
plot_predictions(df, 'sub14_EP09_06inception_facial_lanmarks')

In [8]:
cap = cv2.VideoCapture(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\datasets\CASME2\CASME2\CASME2-RAW\CASME2-RAW\sub09\EP15_05.avi")
if not cap.isOpened():
    raise IOError("Cannot open webcam")

In [9]:
predict_emotion('sub09_EP15_05')

In [24]:
df = pd.read_csv('sub09_EP15_05inception_mp.csv')

In [25]:
df.loc[48]

Confidence                                                 0.581596
max_confidence    [0.5815956592559814, 0.3995557427406311, 0.018...
Emotion                                                    Negative
Name: 48, dtype: object

In [21]:
plot_predictions(df, 'sub09_EP15_05inception_facial_lanmarks')

: 